In [1]:
import os
import sys
import flowfairy
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as display

In [2]:
sys.path.append('..')
os.environ.setdefault('GLUE_SETTINGS_MODULE', 'settings')

'settings'

## Load some data

In [3]:
folder = '/home/sorson/audio_dataset/preprocessed_22050/train/'
all_data = os.listdir(folder)

def get(i):
    return np.load(os.path.join(folder, all_data[i]))

npz1 = get(0)
npz2 = get(1)

def audio(npz):    
    audio_file = str(npz['audio_file']).replace('//','/')
    sound, _ = sf.read(audio_file)
    return sound

a1 = audio(npz1)
a2 = audio(npz2)

display.display(display.Audio(a1, rate=22050))

## Setup the network

In [4]:
import netops

outputlen = min(a1.shape[0], a2.shape[0])

a1 = a1[None, :outputlen, None]
a2 = a2[None, :outputlen, None]

outputlen -= 22050 // 3400 + 10
# decrement outputlen until compatible with network
while outputlen % 4:
    outputlen -= 1

shape = list(a1.shape)

X = tf.placeholder(tf.float32, shape=shape)
X_noise = tf.placeholder(tf.float32, shape=shape)
y = tf.placeholder(tf.int64, shape=shape)
cls = tf.placeholder(tf.int32, shape=[None])

feed_dict = {X: a1, X_noise: a2}

net = netops.Net()

#with tf.device('/cpu:0'):
with tf.variable_scope('network') as scope:
    with tf.name_scope('training'):
        tx, tnet, *_ = net.feedforward(X, X_noise, y, cls, is_training=True, outputlen=outputlen)
    scope.reuse_variables()

    with tf.name_scope('validation'):
        vx, vnet, *_ = net.feedforward(X, X_noise, y, cls, outputlen=outputlen)

x Tensor("network/training/concat_2:0", shape=(1, 291704, 2), dtype=float32)
conv1 Tensor("network/training/glu_conv1_1/mul:0", shape=(1, 291704, 1, 4), dtype=float32)
conv1_d1  Tensor("network/training/glu_conv1_d1/mul:0", shape=(1, 291704, 1, 4), dtype=float32)
conv1_d2  Tensor("network/training/glu_conv1_d2/mul:0", shape=(1, 291704, 1, 4), dtype=float32)
conv1_d4  Tensor("network/training/glu_conv1_d4/mul:0", shape=(1, 291704, 1, 4), dtype=float32)
conv1_concat Tensor("network/training/concat_4:0", shape=(1, 291704, 1, 12), dtype=float32)
d1 Tensor("network/training/dense_1/d_4/glu_glu/mul:0", shape=(1, 291704, 1, 8), dtype=float32)
d2 Tensor("network/training/dense_2/d_4/glu_glu/mul:0", shape=(1, 145852, 1, 16), dtype=float32)
d3 Tensor("network/training/dense_3/d_4/glu_glu/mul:0", shape=(1, 72926, 1, 32), dtype=float32)
d4 Tensor("network/training/dense_4/d_4/glu_glu/mul:0", shape=(1, 145852, 1, 16), dtype=float32)
d5 Tensor("network/training/dense_5/d_4/glu_glu/mul:0", shape=(1, 

## Start a session

In [5]:
session = tf.InteractiveSession()

## Restore the saved network

In [6]:
saver = tf.train.Saver()
network_name = 'denseU_ss_g2.checkpoint'

saver.restore(session, tf.train.latest_checkpoint('../logs', latest_filename=network_name))

INFO:tensorflow:Restoring parameters from /home/s123859/flowfairy/examples/sine_fix/logs/denseU_ss_g2.ckpt-27000


## Run the network on the data

In [7]:
def norm(tensor):
    tmin = tf.reduce_min(tensor)
    return tf.div((tensor - tmin), (tf.reduce_max(tensor) - tmin) + 1e-12)


pred = tf.argmax(tnet, 2)
pred = norm(tf.cast(pred, tf.float32))

In [16]:
noisy, prediction = session.run([tx, pred], feed_dict=feed_dict)

In [17]:
print(prediction[0].shape)
print(noisy.shape)

(291704,)
(1, 291704, 2)


In [18]:

display.display(display.Audio(data=prediction[0], rate=22050))

In [19]:
display.display(display.Audio(data=noisy[:,:,0], rate=22050))

In [20]:
session.close()